# SQL query from table names - Continued

In [2]:
from openai import OpenAI
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

OPENAI_API_KEY  = os.getenv('OPENAI_API_KEY')

## The old Prompt

In [3]:
#The old prompt
old_context = [ {'role':'system', 'content':"""
you are a bot to assist in create SQL commands, all your answers should start with \
this is your SQL, and after that an SQL that can do what the user request. \
Your Database is composed by a SQL database with some tables. \
Try to maintain the SQL order simple.
Put the SQL command in white letters with a black background, and just after \
a simple and concise text explaining how it works.
If the user ask for something that can not be solved with an SQL Order \
just answer something nice and simple, maximum 10 words, asking him for something that \
can be solved with SQL.
"""} ]

old_context.append( {'role':'system', 'content':"""
first table:
{
  "tableName": "employees",
  "fields": [
    {
      "nombre": "ID_usr",
      "tipo": "int"
    },
    {
      "nombre": "name",
      "tipo": "varchar"
    }
  ]
}
"""
})

old_context.append( {'role':'system', 'content':"""
second table:
{
  "tableName": "salary",
  "fields": [
    {
      "nombre": "ID_usr",
      "type": "int"
    },
    {
      "name": "year",
      "type": "date"
    },
    {
      "name": "salary",
      "type": "float"
    }
  ]
}
"""
})

old_context.append( {'role':'system', 'content':"""
third table:
{
  "tablename": "studies",
  "fields": [
    {
      "name": "ID",
      "type": "int"
    },
    {
      "name": "ID_usr",
      "type": "int"
    },
    {
      "name": "educational_level",
      "type": "int"
    },
    {
      "name": "Institution",
      "type": "varchar"
    },
    {
      "name": "Years",
      "type": "date"
    }
    {
      "name": "Speciality",
      "type": "varchar"
    }
  ]
}
"""
})

## New Prompt.
We are going to improve it following the instructions of a Paper from the Ohaio University: [How to Prompt LLMs for Text-to-SQL: A Study in Zero-shot, Single-domain, and Cross-domain Settings](https://arxiv.org/abs/2305.11853). I recommend you read that paper.

For each table, we will define the structure using the same syntax as in a SQL create table command, and add the sample rows of the content.

Finally, at the end of the prompt, we'll include some example queries with the SQL that the model should generate. This technique is called Few-Shot Samples, in which we provide the prompt with some examples to assist it in generating the correct SQL.


In [26]:
context = [ {'role':'system', 'content':"""
    Generate a SQL query that answers the question below.
    This query will run on a database whose schema is represented in this string:

    create table employees(
        ID_Usr INT primary key,-- Unique Id for employee
        name VARCHAR -- Name of employee
        );

    create table salary(
        ID_Usr INT,-- Unique Id for employee
        year DATE, -- Date
        salary FLOAT, --Salary of employee
        foreign key (ID_Usr) references employees(ID_Usr) -- Join Employees with salary
        );

    create table studies(
        ID_study INT, -- Unique ID study
        ID_Usr INT, -- ID employee
        educational_level INT,  -- 5=phd, 4=Master, 3=Bachelor
        Institution VARCHAR, --Name of instituon where eployee studied
        Years DATE, -- Date acomplishement stdy
        Speciality VARCHAR, -- Speciality of studies
        primary key (ID_study, ID_Usr), --Primary Key ID_Usr + ID_Study
        foreign key(ID_Usr) references employees (ID_Usr)
        );

    ### Response
    Based on your instructions, here is the SQL query I have generated to answer the question

"""} ]



In [47]:
#FEW SHOT SAMPLES
context.append( {'role':'system', 'content':"""
 -- Maintain the SQL order simple and efficient as you can, using valid SQL Lite, answer the following questions for the table provided above.
For example:
SELECT COUNT(*) AS total_employees
FROM employees;
"""
})

In [48]:
#Functio to call the model.
def return_CCRMSQL(user_message, context):
    client = OpenAI(
    # This is the default and can be omitted
    api_key=OPENAI_API_KEY,
)

    newcontext = context.copy()
    newcontext.append({'role':'user', 'content':"question: " + user_message})

    response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=newcontext,
            temperature=0,
        )

    return (response.choices[0].message.content)

## NL2SQL Samples
We're going to review some examples generated with the old prompt and others with the new prompt.

In [49]:
#new
print(return_CCRMSQL("How many emloyees do I have?", context_user))

```sql
SELECT COUNT(*) AS total_employees
FROM employees;
```


In [50]:
#old
old_context_user = old_context.copy()
print(return_CCRMSQL("How many emloyees do I have?", old_context_user))

This is your SQL:
```sql
SELECT COUNT(*) AS total_employees FROM employees;
```

This SQL command will count the total number of employees in the "employees" table and return the result as "total_employees".


In [39]:
#new
print(return_CCRMSQL("How much salary do I pay?", context_user))

SELECT SUM(salary) AS total_salary_paid
FROM salary;


In [40]:
#old
old_context_user = old_context.copy()
print(return_CCRMSQL("How much salary do I pay?", old_context_user))

This is your SQL:
```sql
SELECT SUM(salary) AS total_salary_paid
FROM salary;
```

This SQL query calculates the total amount of salary paid by summing up all the values in the "salary" column from the "salary" table.


In [44]:
#new
print(return_CCRMSQL("What did my employees study?", context_user))

SELECT e.name, s.Speciality
FROM employees e
JOIN studies st ON e.ID_Usr = st.ID_Usr;


In [45]:
#old
print(return_CCRMSQL("What did my employees study?", old_context_user))

This is your SQL:
```sql
SELECT employees.name, studies.educational_level, studies.Institution, studies.Years, studies.Speciality
FROM employees
JOIN studies ON employees.ID_usr = studies.ID_usr;
```

This SQL query retrieves the names of employees along with their educational level, institution, years of study, and specialty by joining the "employees" and "studies" tables on the common ID_usr field.


# Exercise
 - Complete the prompts similar to what we did in class. 
     - Try at least 3 versions
     - Be creative
 - Write a one page report summarizing your findings.
     - Were there variations that didn't work well? i.e., where GPT either hallucinated or wrong.
     - What did you learn?